<a href=\"https://colab.research.google.com/github/bearpelican/musicautobot/blob/master/notebooks/music_transformer/Generate_colab.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>

# Feed Forward Net

Im folgenden wollen wir eines der einfachsten neuronalen Netze für die Melodiegenerierung verwenden: Ein Netwerk mit lediglich einer Schicht, d.h., die Parameter $\theta$ stecken alle in einer einzigen Matrix $W$.

Gegeben eines Vektors $\mathbf{x}$ der eine Note/Event repräsentiert soll 

$$\mathbf{x}^{\top} W$$

die nächste Note/Event ergeben.

In [ ]:
import sys
sys.path.append("..") 

import matplotlib.pyplot as plt
import music21 as m21
import torch
import torch.nn.functional as F
from preprocess import load_songs_in_kern, NoteEncoder, StringToIntEncoder, TERM_SYMBOL

torch.manual_seed(0);

Da wir diesesmal unser Netzwerk trainieren, verwenden wir die GPU, sofern dies möglich ist:

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
    
print(f'{device=}')

# 1. Datenvorbereitung (wie zuvor)

Zunächst laden wir unsere (1700 **einstimmigen**) Musikstücke aus denen wir die Frequenzen berechnen wollen.

In [ ]:
scores = load_songs_in_kern('./../deutschl/erk')

Wir können uns eines der Musikstücke anhören oder auch anzeigen lassen.

In [ ]:
scores[0].show('midi')

In [ ]:
scores[0].show()

Als nächstes verwandeln wir die Noten in gut leserliche Zeichenketten, wobei jede Note durch genau eine Zeichenkette repräsentiert wird und zwar der Form ``MIDI-Note/Länge in vielfaches von time_step``.

Dies übernimmt der ``NoteEncoder``. Dieser transponiert die Musikstücke zusätzlich nach C-Dur.
Dieser filtert zugleich Musikstücke heraus, welche wir mit unserem ``time_step`` nicht abbilden können.
Z.B. wenn ``time_step = 1/8`` dann können wir keine ``1/16``-Noten oder auch ``1/8 + 1/16``-Noten abbilden. 

In [ ]:
time_step = 1/8
print(f'one timestep represents {time_step} beats')

encoder = NoteEncoder(time_step)
enc_songs, invalid_song_indices = encoder.encode_songs(scores)

print(f'there are {len(enc_songs)} valid songs and {len(invalid_song_indices)} songs')

In [ ]:
' '.join(enc_songs[0])

In [ ]:
print(f'longest melody: {max(len(m) for m in enc_songs)}')
print(f'shortest melody: {min(len(m) for m in enc_songs)}')

Da der Computer besser mit Zahlen umgehen kann bauen wir uns eine Abbildung von den jeweiligen Zeichenketten zu Zahlen $$\{0, 1, 2, \ldots, m-1\}$$ und umgekehrt. Dies übernimmt ``StringToIntEncoder``:

In [ ]:
string_to_int = StringToIntEncoder(enc_songs)
print(f'number of unique symbols: {len(string_to_int)}')

In [ ]:
scores[0].show('midi')

# 2. Konstruktion der Trainingsdaten

Statt die Übergänge direkt zu zählen, ist jeder Notenübergang von $e_i$ nach $e_j$.

$\mathbf{x}$ wird durch einen Vektor repräsentiert der $m$ Komponenten besitzt wobei $m$ die Anzahl an verschiedenen Noten ist. $\mathbf{x}$ hat genau einen Eintrag der 1 ist alle anderen sind 0. Diese Art der Codierung nennt sich *one-hot Encoding*.

$X$ ist eine Matrix die alle $\mathbf{x}$ als Zeilenvektoren enthält.

In [ ]:
m = len(string_to_int)

xs = []
ys = []
for enc_song in enc_songs:
    chs = [TERM_SYMBOL] + enc_song + [TERM_SYMBOL]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = string_to_int.encode(ch1)
        ix2 = string_to_int.encode(ch2)
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs, device=device)
y = torch.tensor(ys, device=device)

# one-hot-encoding
# macht aus z.B. aus [0,2,1,3,2] den tensor
# [[0,0,0,1],[0,1,0,0],[0,0,1,0],[1,0,0,0],[0,1,0,0]]
X = F.one_hot(xs, num_classes=m).float()

In [ ]:
W = torch.randn((m, m), requires_grad=True, device=device)
print(f'matrix W has the shape {W.shape}')

# 3. Training

Der Forwardpass berechnet sich wie folgt: Zuerst multiplizieren wir die beiden Matrizen 

$$C = X \cdot W.$$

Dann berechnen **zeilenweise** die sog. *softmax* operation.

$$\frac{\exp(c_k)}{\sum_{j=1} \exp(c_j) }$$

oder anders ausgedrückt **normieren** wir jede Zeile nachdem wir komponentenweise die exponentialfunktion angewendet haben. Eine Zeile der resultierenden Matrix $P$ kann somit als Wahrscheinlichkeitsverteilung interpretiert werden!

Für die Optimierung durch *Gradient Decent* benötigen wir eine geeignete *Kostenfunktion*/*Lossfunction* $L$.
Dazu betrachten wir jene "Wahrscheinlichkeit" für alle richtig gewählten Übergänge, d.h. die *Likelihood*.
Seien $p_1, \ldots, p_n$ diese Wahrscheinlichkeiten (eine pro Übergäng aka Zeile in $X$) dann ist 

$$p_1 \cdot \ldots \cdot p_n$$

die *Likelihood*.
Um stattdessen addieren zu können berechnen wir jedoch die *negative log Likelihood*:

$$-(\log(p_1) + \ldots + \log(p_n).$$

Durch den Aufruf ``loss.backward()`` wird die *Backpropagation* (auch *Backwardpass*) durchgeführt und wir können unsere Gewichte durch

$$W \leftarrow W - \eta \cdot \nabla_W L $$

In unserem Fall wählen wir eine sehr große *Lernrate* $\eta$.

In [ ]:
# training aka gradient decent
epochs = 500
for k in range(epochs):
    # 1. forward pass
    logits = X @ W
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(len(ys), device=device), y].log().mean()
    
    if k % 100 == 0:
        print(f'epoch {k}, loss: {loss.item()}')
    
    # 2. backward pass
    W.grad = None # set gradients to zero
    loss.backward()
    W.data += -10.0 * W.grad  

# 4. Melodiegenerierung  (wie zuvor)

Wie zuvor allerdings müssen wir $P$ aus $W$ berechnen. **Achtung:** $P$ ist nicht gleich der *Markov-Matrix* von zuvor allerdings konvergiert es gegen diese.

In [ ]:
P = W.exp() / W.exp().sum(dim=1, keepdim=True)

Mit $P$ können wir nun neue Melodien generieren.
Wir starten mit dem speziellen Symbol $e_1$ = ``TERM_SYMBOL`` ("Beginn des Stücks") als unsere erste Note/Event $e_1$. Dann berechnen wir $e_2$ aus der bedingten Verteilung $P(X_2 = e_2 | X_1 = e_1)$, gegeben durch die Zeile die zu $e_1$ gehört, ziehen. Als nächstes berechnen wir $e_3$ durch $P(X_2 = e_3 | X_1 = e_2)$.

Diesen Vorgang führen wir so lange fort bis wir irgendwann ``TERM_SYMBOL`` ziehen.
Sie können eine maximale Länge des Stücks festlegen und auch einen Anfang eines Stücks mitliefern.

``temperature`` bestimmt wie stark die vom Modell gelernte Wahrscheinlichkeitsverteilung beachtet wird.

+ ``temperature`` gleich 1.0 bedeutet, dass von der Wahrscheinlichkeitsverteilung gesampelt wird.
+ ``temperature`` gegen unendlich bedeutet, dass gleichverteilt gesampelt wird (mehr Variation)
+ ``temperature`` gegen 0 bedeutet, dass die hohe Wahrscheinlichkeiten verstärkt werden (weniger Variation)

In [ ]:
def next_event_number(char:str, temperature:float):
    with torch.no_grad():
        distribution = P[string_to_int.encode(char)] / temperature
        ix = torch.multinomial(distribution, num_samples=1, replacement=True).item()
        return ix

In [ ]:
def generate(seq: list[str]=None, max_len:int=None, temperature:float=1.0):
    generated_encoded_song = []
    if seq != None:
        generated_encoded_song = seq.copy()
    char = TERM_SYMBOL
    while max_len == None or max_len > len(generated_encoded_song):
        ix = next_event_number(char, temperature)
        char = string_to_int.decode(ix)
        if char == TERM_SYMBOL:
            break
        generated_encoded_song.append(char)
    return generated_encoded_song

In [ ]:
# number of songs we want to generate
n_scores = 5
generated_encoded_songs = []

for _ in range(n_scores):
    encoded_song = generate()
    print(f'generated {" ".join(encoded_song)} conisting of {len(encoded_song)} notes')
    generated_encoded_songs.append(encoded_song)

# 5. Datennachbearbeitung (wie zuvor)

Wir wollen uns die generierten Stücke natürlich anhören. Der ``NoteEncoder`` kann dies für uns übernehmen.

In [ ]:
generated_scores = encoder.decode_songs(generated_encoded_songs)

In [ ]:
generated_scores[0].show('midi')

In [ ]:
generated_scores[0].show()